In [ ]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.stats import entropy
import pandas as pd
import zipfile

# Set seed for reproducibility
random.seed("This is a seed.")

# ============ PATH CONFIGURATION ============
base_path = "../Photos_Subset/Original"

# ============ VERIFY PATHS ============
if not os.path.exists(base_path):
    raise FileNotFoundError(f"Original folder not found at {base_path}")

if not os.path.isdir(base_path):
    raise NotADirectoryError(f"{base_path} is not a directory")

print(f" Base path verified: {base_path}")
print(f" Random seed: 'This is a seed.'\n")


# ---------------------------- Noise functions with parameters ----------------------------
def add_salt_pepper_noise(image, amount=0.2, salt_vs_pepper=0.5):
    """
    Add salt and pepper noise
    amount: proportion of image to be noised (0 to 1)
    salt_vs_pepper: proportion of salt vs pepper (0.5 = equal amounts)
    """
    noisy = np.copy(image)
    num_salt = np.ceil(amount * image.size * salt_vs_pepper)
    num_pepper = np.ceil(amount * image.size * (1 - salt_vs_pepper))

    # Add salt (white pixels)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = 255

    # Add pepper (black pixels)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = 0

    return noisy

def add_speckle_noise(image, intensity=1.0):
    """
    Add speckle noise
    intensity: multiplier for noise strength (typical range: 0.1 to 5.0)
    """
    row, col, ch = image.shape
    gauss = np.random.randn(row, col, ch)
    noisy = image + image * gauss * intensity
    return np.clip(noisy, 0, 255).astype(np.uint8)

# ---------------------------- Comprehensive Metric Calculation ----------------------------
def calculate_entropy(image):
    """Calculate entropy of image"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    hist, _ = np.histogram(gray.flatten(), bins=256, range=(0, 256))
    hist = hist / hist.sum()  # Normalize
    hist = hist[hist > 0]  # Remove zeros
    return entropy(hist, base=2)

def calculate_sharpness(image):
    """Calculate sharpness using Laplacian variance"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    return laplacian.var()

def calculate_spatial_frequency(image):
    """Calculate spatial frequency"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    gray = gray.astype(np.float64)

    # Row frequency
    RF = np.sqrt(np.mean(np.diff(gray, axis=1) ** 2))

    # Column frequency
    CF = np.sqrt(np.mean(np.diff(gray, axis=0) ** 2))

    # Spatial frequency
    SF = np.sqrt(RF ** 2 + CF ** 2)

    return SF

def calculate_dynamic_range(image):
    """Calculate dynamic range"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    return np.max(gray) - np.min(gray)

def calculate_noise_variance(original, noisy):
    """Calculate variance of the noise (difference between original and noisy)"""
    if len(original.shape) == 3:
        original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
        noisy_gray = cv2.cvtColor(noisy, cv2.COLOR_BGR2GRAY)
    else:
        original_gray = original
        noisy_gray = noisy

    noise = noisy_gray.astype(np.float64) - original_gray.astype(np.float64)
    return np.var(noise)

def calculate_all_metrics(original, noisy):
    """Calculate all metrics between original and noisy images"""
    # Convert to grayscale for metric calculation
    if len(original.shape) == 3:
        original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
        noisy_gray = cv2.cvtColor(noisy, cv2.COLOR_BGR2GRAY)
    else:
        original_gray = original
        noisy_gray = noisy

    # PSNR
    psnr_value = psnr(original_gray, noisy_gray, data_range=255)

    # SSIM
    ssim_value = ssim(original_gray, noisy_gray, data_range=255)

    # MSE
    mse_value = np.mean((original_gray.astype(np.float64) - noisy_gray.astype(np.float64)) ** 2)

    # Entropy difference
    entropy_orig = calculate_entropy(original)
    entropy_noisy = calculate_entropy(noisy)
    entropy_diff = abs(entropy_noisy - entropy_orig)

    # Noise variance
    noise_var = calculate_noise_variance(original, noisy)

    # Sharpness
    sharpness_value = calculate_sharpness(noisy)

    # Spatial frequency
    spatial_freq = calculate_spatial_frequency(noisy)

    # Dynamic range
    dynamic_range = calculate_dynamic_range(noisy)

    return {
        "PSNR": psnr_value,
        "SSIM": ssim_value,
        "MSE": mse_value,
        "Entropy_Diff": entropy_diff,
        "Noise_Variance": noise_var,
        "Sharpness": sharpness_value,
        "Spatial_Freq": spatial_freq,
        "Dynamic_Range": dynamic_range
    }

# ---------------------------- Parameter configurations ----------------------------
# Different parameter sets to test
SALT_PEPPER_PARAMS = [
    {"amount": 0.01, "salt_vs_pepper": 0.5, "name": "amount_0.01"},
    {"amount": 0.05, "salt_vs_pepper": 0.5, "name": "amount_0.05"},
    {"amount": 0.1, "salt_vs_pepper": 0.5, "name": "amount_0.10"},
    {"amount": 0.15, "salt_vs_pepper": 0.5, "name": "amount_0.15"},
    {"amount": 0.2, "salt_vs_pepper": 0.5, "name": "amount_0.20"},
]

SPECKLE_PARAMS = [
    {"intensity": 0.5, "name": "intensity_0.5"},
    {"intensity": 1.0, "name": "intensity_1.0"},
    {"intensity": 1.5, "name": "intensity_1.5"},
    {"intensity": 2.0, "name": "intensity_2.0"},
    {"intensity": 2.5, "name": "intensity_2.5"},
    {"intensity": 3.0, "name": "intensity_3.0"},
]

# ---------------------------- Verify base_path exists ----------------------------
if not os.path.exists(base_path):
    raise FileNotFoundError(f"Base path does not exist: {base_path}")
if not os.path.isdir(base_path):
    raise NotADirectoryError(f"Base path is not a directory: {base_path}")

print(f"Base path verified: {base_path}")
print(f'Random seed set to: "This is a seed."')

# ---------------------------- Folder setup ----------------------------
noised_root = os.path.join(base_path, "Noised_Images_Params")
metrics_root = os.path.join(base_path, "Metrics_CSVs")

os.makedirs(noised_root, exist_ok=True)
os.makedirs(metrics_root, exist_ok=True)

# Get all image files and separate them into color and BW
all_files = [f for f in os.listdir(base_path)
             if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))
             and os.path.isfile(os.path.join(base_path, f))]

# Separate color and BW files similar to teammate's approach
color_files = []
bw_files = []

for filename in all_files:
    if filename[-6:-4] == 'bw' or filename.lower().endswith('_bw.jpg') or filename.lower().endswith('_bw.png'):
        bw_files.append(filename)
    else:
        color_files.append(filename)

print(f"Found {len(color_files)} color images")
print(f"Found {len(bw_files)} BW images")
print(f"Total: {len(all_files)} images")

# ---------------------------- Process Salt & Pepper Noise ----------------------------


for param_config in SALT_PEPPER_PARAMS:
    param_name = param_config["name"]
    amount = param_config["amount"]
    salt_vs_pepper = param_config["salt_vs_pepper"]

    print(f"\nProcessing: {param_name} (amount={amount}, salt_vs_pepper={salt_vs_pepper})")

    # Create output directory for this parameter
    param_output_dir = os.path.join(noised_root, param_name)
    os.makedirs(param_output_dir, exist_ok=True)

    # Store metrics for this parameter configuration
    metrics_color = []
    metrics_bw = []

    # Process all files (color and BW together)
    print(f"  Processing images...")
    for filename in tqdm(all_files, desc=f"    {param_name}"):
        filepath = os.path.join(base_path, filename)
        img = cv2.imread(filepath)

        if img is None:
            continue

        # Add noise
        noisy_img = add_salt_pepper_noise(img, amount=amount, salt_vs_pepper=salt_vs_pepper)

        # Save with same naming convention as teammate
        output_filename = f"noisy_{filename}"
        output_path = os.path.join(param_output_dir, output_filename)
        cv2.imwrite(output_path, noisy_img)

        # Calculate metrics
        metrics = calculate_all_metrics(img, noisy_img)
        metrics["Name"] = output_filename

        # Separate metrics by color vs BW
        if filename[-6:-4] == 'bw' or filename.lower().endswith('_bw.jpg') or filename.lower().endswith('_bw.png'):
            metrics_bw.append(metrics)
        else:
            metrics_color.append(metrics)

    # Save CSV for this parameter configuration - COLOR
    if metrics_color:
        df_color = pd.DataFrame(metrics_color)
        column_order = ["Name", "PSNR", "SSIM", "MSE", "Entropy_Diff",
                        "Noise_Variance", "Sharpness", "Spatial_Freq", "Dynamic_Range"]
        df_color = df_color[column_order]
        csv_path_color = os.path.join(metrics_root, f"noisy_{param_name}.csv")
        df_color.to_csv(csv_path_color, index=False)
        print(f"  Saved: {csv_path_color}")

    # Save CSV for this parameter configuration - BW
    if metrics_bw:
        df_bw = pd.DataFrame(metrics_bw)
        df_bw = df_bw[column_order]
        csv_path_bw = os.path.join(metrics_root, f"noisy_{param_name}_bw.csv")
        df_bw.to_csv(csv_path_bw, index=False)
        print(f"  Saved: {csv_path_bw}")

# ---------------------------- Process Speckle Noise ----------------------------

for param_config in SPECKLE_PARAMS:
    param_name = param_config["name"]
    intensity = param_config["intensity"]

    print(f"\nProcessing: {param_name} (intensity={intensity})")

    # Create output directory for this parameter
    param_output_dir = os.path.join(noised_root, param_name)
    os.makedirs(param_output_dir, exist_ok=True)

    # Store metrics for this parameter configuration
    metrics_color = []
    metrics_bw = []

    # Process all files (color and BW together)
    print(f"  Processing images...")
    for filename in tqdm(all_files, desc=f"    {param_name}"):
        filepath = os.path.join(base_path, filename)
        img = cv2.imread(filepath)

        if img is None:
            continue

        # Add noise
        noisy_img = add_speckle_noise(img, intensity=intensity)

        # Save with same naming convention as teammate
        output_filename = f"noisy_{filename}"
        output_path = os.path.join(param_output_dir, output_filename)
        cv2.imwrite(output_path, noisy_img)

        # Calculate metrics
        metrics = calculate_all_metrics(img, noisy_img)
        metrics["Name"] = output_filename

        # Separate metrics by color vs BW
        if filename[-6:-4] == 'bw' or filename.lower().endswith('_bw.jpg') or filename.lower().endswith('_bw.png'):
            metrics_bw.append(metrics)
        else:
            metrics_color.append(metrics)

    # Save CSV for this parameter configuration - COLOR
    if metrics_color:
        df_color = pd.DataFrame(metrics_color)
        column_order = ["Name", "PSNR", "SSIM", "MSE", "Entropy_Diff",
                        "Noise_Variance", "Sharpness", "Spatial_Freq", "Dynamic_Range"]
        df_color = df_color[column_order]
        csv_path_color = os.path.join(metrics_root, f"noisy_{param_name}.csv")
        df_color.to_csv(csv_path_color, index=False)
        print(f"  Saved: {csv_path_color}")

    # Save CSV for this parameter configuration - BW
    if metrics_bw:
        df_bw = pd.DataFrame(metrics_bw)
        df_bw = df_bw[column_order]
        csv_path_bw = os.path.join(metrics_root, f"noisy_{param_name}_bw.csv")
        df_bw.to_csv(csv_path_bw, index=False)
        print(f"  Saved: {csv_path_bw}")


print(f"\nTotal parameter configurations tested:")
print(f"  Salt & Pepper: {len(SALT_PEPPER_PARAMS)} configurations")
print(f"  Speckle: {len(SPECKLE_PARAMS)} configurations")
print(f"  Total CSV files created: {(len(SALT_PEPPER_PARAMS) + len(SPECKLE_PARAMS)) * 2}")
print(f"\nImages processed:")
print(f"  Color: {len(color_files)}")
print(f"  BW: {len(bw_files)}")
print(f"\nAll metrics saved to: {metrics_root}")
print(f"All noised images saved to: {noised_root}")
print("\nDone")

FileNotFoundError: Original folder not found at ./Photos_Subset/Original